In [7]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from gurobipy import Model, GRB, quicksum

# QUESTION 1

In [ ]:
capacity = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/capacity.csv')
costs_revenues = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/costs_revenues.csv')
demand = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/demand.csv')

# QUESTION 2

In [ ]:
costs = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/costs.csv')
randomness = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/randomness.csv')

In [4]:
costs

,Unnamed: 0,Station_0,Station_1,Station_2,Station_3,Station_4,Station_5,Station_6,Station_7,Station_8,Station_9,Station_10,Station_11,Station_12,Station_13,Station_14
0,Station_0,0,43,7,27,20,22,43,28,42,30,22,27,36,35,16
1,Station_1,22,0,12,38,19,14,46,13,30,47,18,29,11,32,9
2,Station_2,23,46,0,14,43,14,33,23,13,45,12,12,28,16,23
3,Station_3,10,45,14,0,44,35,29,25,5,29,46,46,25,34,21
4,Station_4,39,32,15,44,0,24,48,47,25,34,14,47,45,10,23
5,Station_5,27,14,44,23,8,0,24,39,16,40,13,22,19,17,11
6,Station_6,49,10,27,25,40,16,0,10,46,26,30,15,47,19,10
7,Station_7,27,18,32,8,37,40,29,0,42,19,41,20,40,9,23
8,Station_8,14,37,8,14,44,43,23,5,0,33,47,12,39,33,26
9,Station_9,27,5,27,48,47,6,39,22,17,0,40,42,15,22,23


In [5]:
randomness

,Unnamed: 0,Probability,Mean_Demand,Std_Dev_Demand
0,Station_1,0.549406,184,29
1,Station_2,0.597865,310,33
2,Station_3,0.274362,465,14
3,Station_4,0.204316,193,39
4,Station_5,0.241876,118,43
5,Station_6,0.703453,265,33
6,Station_7,0.699967,169,13
7,Station_8,0.780134,273,46
8,Station_9,0.823935,393,43
9,Station_10,0.709039,308,33


2. e) Create a stochastic program using a Monte Carlo simulation of 20 trials with 10 scenarios per trial (i.e., use SAA). What is the optimal expected cost?

In [13]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from gurobipy import Model, GRB, quicksum

# Load data
costs = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/costs.csv', index_col=0)
randomness = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/randomness.csv')

# Fix labels and ensure all values are numeric
costs.index = [f"Station_{i}" for i in range(len(costs.index))]
costs.columns = [f"Station_{i}" for i in range(len(costs.columns))]
costs = costs.apply(pd.to_numeric, errors='coerce')  # ensures all values are numeric

# Parameters
n_trials = 20
n_scenarios = 10
over_cost = 0.09
under_cost = 0.13

# Generate a random demand scenario
def generate_scenario():
    scenario = {}
    for _, row in randomness.iterrows():
        station = row['Unnamed: 0']
        if np.random.rand() < row['Probability']:
            demand = np.random.normal(row['Mean_Demand'], row['Std_Dev_Demand'])
            scenario[station] = max(0, demand)
    return scenario

# Estimate cost for a given scenario using naive truck size and route
def solve_scenario(demand_scenario):
    total_demand = sum(demand_scenario.values())
    truck_size = total_demand  # naive: match the day's demand

    # Penalty costs
    underage = max(0, total_demand - truck_size)
    overage = max(0, truck_size - total_demand)
    penalty_cost = under_cost * underage + over_cost * overage

    # Naive route: from storage to each station and back (chain-style)
    nodes = ['Station_0'] + list(demand_scenario.keys()) + ['Station_0']
    travel_cost = 0
    for i in range(len(nodes) - 1):
        src = nodes[i]
        dst = nodes[i + 1]
        if src != dst:
            travel_cost += costs.loc[src, dst]

    return travel_cost + penalty_cost

# Monte Carlo SAA
expected_costs = []
for _ in range(n_trials):
    trial_costs = []
    for _ in range(n_scenarios):
        scenario = generate_scenario()
        cost = solve_scenario(scenario)
        trial_costs.append(cost)
    expected_costs.append(np.mean(trial_costs))

optimal_expected_cost = np.mean(expected_costs)
print(f"Estimated optimal expected cost over 20 trials: ${optimal_expected_cost:.2f}")

Estimated optimal expected cost over 20 trials: $278.27


2. f) Using a Monte Carlo simulation of 20 trials with 10 scenarios per trial, what is the expected value of reacting with perfect foresight (WS) and the expected value of perfect information (EVPI) associated with this stochastic program?

In [14]:
# Generate a random demand scenario
def generate_scenario():
    scenario = {}
    for _, row in randomness.iterrows():
        station = row['Unnamed: 0']
        if np.random.rand() < row['Probability']:
            demand = np.random.normal(row['Mean_Demand'], row['Std_Dev_Demand'])
            scenario[station] = max(0, demand)
    return scenario

# Estimate cost for a given scenario using naive truck size and route
def solve_scenario(demand_scenario):
    total_demand = sum(demand_scenario.values())
    truck_size = total_demand  # naive: match the day's demand

    # Penalty costs
    underage = max(0, total_demand - truck_size)
    overage = max(0, truck_size - total_demand)
    penalty_cost = under_cost * underage + over_cost * overage

    # Naive route: from storage to each station and back (chain-style)
    nodes = ['Station_0'] + list(demand_scenario.keys()) + ['Station_0']
    travel_cost = 0
    for i in range(len(nodes) - 1):
        src = nodes[i]
        dst = nodes[i + 1]
        if src != dst:
            travel_cost += costs.loc[src, dst]

    return travel_cost + penalty_cost


# Redefine solve_perfect_foresight: optimize based on known demand
def solve_perfect_foresight(scenario):
    # Route and truck size are customized to known demand
    total_demand = sum(scenario.values())
    truck_size = total_demand  # perfect foresight, no overage or underage
    penalty_cost = 0  # no penalty since truck size matches demand

    # Same naive routing logic as before
    nodes = ['Station_0'] + list(scenario.keys()) + ['Station_0']
    travel_cost = 0
    for i in range(len(nodes) - 1):
        src = nodes[i]
        dst = nodes[i + 1]
        if src != dst:
            travel_cost += costs.loc[src, dst]
    
    return travel_cost + penalty_cost

# Run Monte Carlo for WS value
ws_costs = []
for _ in range(n_trials):
    trial_costs = []
    for _ in range(n_scenarios):
        scenario = generate_scenario()
        cost = solve_perfect_foresight(scenario)
        trial_costs.append(cost)
    ws_costs.append(np.mean(trial_costs))

expected_WS = np.mean(ws_costs)
EVPI = expected_WS - optimal_expected_cost  # optimal_expected_cost is from part (e)

print(f"Expected Wait-and-See (WS) Cost: ${expected_WS:.2f}")
print(f"Expected Value of Perfect Information (EVPI): ${EVPI:.2f}")

Expected Wait-and-See (WS) Cost: $280.02
Expected Value of Perfect Information (EVPI): $1.75


2. g) Using a Monte Carlo simulation of 20 trials with 10 scenarios per trial, what is the expected value of using the solution from the mean value problem (EEV) and the value of the stochastic solution (VSS) associated with this stochastic program? To compute the EV solution, assume that the fuel truck visits every customer and satisfies their mean demand.

In [15]:
# Step 1: Define the EV solution (visit all, use mean demand as truck size)
mean_demands = randomness.set_index('Unnamed: 0')['Mean_Demand'].to_dict()
ev_truck_size = sum(mean_demands.values())
ev_route = ['Station_0'] + list(mean_demands.keys()) + ['Station_0']

def solve_EEV_using_fixed_truck(ev_truck_size, scenario):
    # Penalty cost based on mismatch with realized total demand
    total_demand = sum(scenario.values())
    underage = max(0, total_demand - ev_truck_size)
    overage = max(0, ev_truck_size - total_demand)
    penalty_cost = under_cost * underage + over_cost * overage

    # Travel cost using fixed route
    travel_cost = 0
    for i in range(len(ev_route) - 1):
        src = ev_route[i]
        dst = ev_route[i + 1]
        if src != dst:
            travel_cost += costs.loc[src, dst]

    return travel_cost + penalty_cost

# Step 2: Run EEV simulation over 20 trials x 10 scenarios
eev_costs = []
for _ in range(n_trials):
    trial_costs = []
    for _ in range(n_scenarios):
        scenario = generate_scenario()
        cost = solve_EEV_using_fixed_truck(ev_truck_size, scenario)
        trial_costs.append(cost)
    eev_costs.append(np.mean(trial_costs))

expected_EEV = np.mean(eev_costs)
VSS = expected_EEV - optimal_expected_cost  # optimal_expected_cost from part (e)

print(f"Expected cost of EEV solution: ${expected_EEV:.2f}")
print(f"Value of the Stochastic Solution (VSS): ${VSS:.2f}")

Expected cost of EEV solution: $613.95
Value of the Stochastic Solution (VSS): $335.68


In [16]:
import pandas as pd

# Create summary dictionary
summary = {
    "Metric": [
        "RP (Stochastic Solution)", 
        "WS (Perfect Foresight)", 
        "EVPI", 
        "EEV (Mean Value Solution)", 
        "VSS"
    ],
    "Value ($)": [
        round(optimal_expected_cost, 2), 
        round(expected_WS, 2), 
        round(EVPI, 2), 
        round(expected_EEV, 2), 
        round(VSS, 2)
    ]
}

# Convert to DataFrame
summary_df = pd.DataFrame(summary)
display(summary_df)

,Metric,Value ($)
0,RP (Stochastic Solution),278.27
1,WS (Perfect Foresight),280.02
2,EVPI,1.75
3,EEV (Mean Value Solution),613.95
4,VSS,335.68
